# TranslateGemma - Document Translator

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jimmyliao/trans-gemma/blob/main/document-translator-colab.ipynb)

Translate PDFs, images, and websites to Traditional Chinese (zh-TW) using Google's TranslateGemma model.

**Features:**
- 📄 Download from arXiv automatically
- 🖼️ Upload images or PDFs
- 🌐 Screenshot websites and translate
- 🚀 Fast GPU inference on Colab (T4)
- 🇹🇼 Force Traditional Chinese output (configurable)

**Single Source of Truth:** Uses the same code from [trans-gemma repo](https://github.com/jimmyliao/trans-gemma)

---

## 👤 About the Author

**Jimmy Liao** - AI GDE (Google Developer Expert), CTO/Co-Founder of AI Startup

Dedicated to smart manufacturing and finance sectors, focusing on transforming technical challenges from AI advancement into competitive advantages while enhancing client value and operational efficiency.

- 🐦 Twitter: [@jimmyliao](https://twitter.com/jimmyliao)
- 💼 LinkedIn: [jimmyliao](https://linkedin.com/in/jimmyliao)
- 📝 Blog: [memo.jimmyliao.net](https://memo.jimmyliao.net)
- 🔗 Sessionize: [jimmy-liao](https://sessionize.com/jimmy-liao/)

---

**Disclaimer:** This notebook is provided for educational and research purposes. The author is not affiliated with Google's TranslateGemma team. Use at your own discretion.

## 1️⃣ Setup: Clone Repository

In [ ]:
# Clean up existing directory if it exists
!rm -rf trans-gemma

# Clone the repository (single source of truth)
!git clone https://github.com/jimmyliao/trans-gemma.git
%cd trans-gemma

## 2️⃣ Install Dependencies

In [ ]:
# Install uv (fast Python package manager)
!pip install uv -q

# Install project dependencies
!uv pip install --system -e ".[examples]"

## 2.5️⃣ HuggingFace Authentication

**IMPORTANT:** TranslateGemma is a gated model. You need to:
1. Get a HuggingFace token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Accept model access at [https://huggingface.co/google/translategemma-4b-it](https://huggingface.co/google/translategemma-4b-it)
3. Add your token to Colab Secrets (🔑 icon on left sidebar, name: `HF_TOKEN`)

In [ ]:
from huggingface_hub import login
import os

try:
    # Try to load from Colab secrets (recommended)
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    os.environ['HF_TOKEN'] = HF_TOKEN
    login(token=HF_TOKEN)
    print("✅ Authenticated with HuggingFace using Colab secrets")
except Exception as e:
    print("⚠️  Could not load HF_TOKEN from Colab secrets")
    print("Please add HF_TOKEN to Colab Secrets (🔑 icon on left sidebar)")
    print(f"Error: {e}")
    
    # Fallback: Manual token input
    print("\n📝 Or enter your token manually:")
    HF_TOKEN = input("HuggingFace Token: ")
    os.environ['HF_TOKEN'] = HF_TOKEN
    login(token=HF_TOKEN)
    print("✅ Authenticated with HuggingFace")

## 3️⃣ Configuration

In [ ]:
import os

# Target language (default: Traditional Chinese)
TARGET_LANG = "zh-TW"  # Change to "zh-CN", "ja", "ko", etc. if needed

# Backend (transformers is best for Colab GPU)
BACKEND = "transformers"

print(f"✅ Target language: {TARGET_LANG}")
print(f"✅ Backend: {BACKEND}")

## 4️⃣ Option A: Download from arXiv

Automatically download and translate arXiv papers.

In [ ]:
# Enter arXiv ID (e.g., "2601.09012v2" or "2601.09012")
ARXIV_ID = "2601.09012v2"  # TranslateGemma technical report

# Translate specific pages (1-indexed)
START_PAGE = 1
END_PAGE = 1  # Set to None for all pages

# Build command
cmd = f"python examples/translate.py --mode pdf --arxiv {ARXIV_ID} --backend {BACKEND} --target {TARGET_LANG}"
if START_PAGE:
    cmd += f" --start-page {START_PAGE}"
if END_PAGE:
    cmd += f" --end-page {END_PAGE}"

print(f"Running: {cmd}\n")
!{cmd}

## 4️⃣ Option B: Upload PDF

Upload your own PDF file.

In [ ]:
from google.colab import files
import os

# Upload PDF
print("📤 Please upload your PDF file:")
uploaded = files.upload()

# Get uploaded filename
pdf_file = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {pdf_file}")

# Translate settings
START_PAGE = 1
END_PAGE = 3  # Change as needed
USE_IMAGE_MODE = False  # Set to True for multimodal (slower but preserves layout)
DPI = 96  # For image mode: lower = faster (72, 96, or 150)

# Build command
cmd = f"python examples/translate.py --mode pdf --file {pdf_file} --backend {BACKEND} --target {TARGET_LANG}"
if START_PAGE:
    cmd += f" --start-page {START_PAGE}"
if END_PAGE:
    cmd += f" --end-page {END_PAGE}"
if USE_IMAGE_MODE:
    cmd += f" --pdf-as-image --dpi {DPI}"

print(f"\nRunning: {cmd}\n")
!{cmd}

## 4️⃣ Option C: PDF with Image Mode (Multimodal)

Use multimodal TranslateGemma to preserve visual context (tables, charts).

In [ ]:
# Translate PDF page with charts/figures (multimodal)
ARXIV_ID = "2601.09012v2"
START_PAGE = 3  # Page with Figure 1 (language distribution charts)
END_PAGE = 3
DPI = 96  # Lower DPI = faster (72, 96, or 150)

cmd = f"python examples/translate.py --mode pdf --arxiv {ARXIV_ID} --backend {BACKEND} --target {TARGET_LANG} --pdf-as-image --dpi {DPI}"
if START_PAGE:
    cmd += f" --start-page {START_PAGE}"
if END_PAGE:
    cmd += f" --end-page {END_PAGE}"

print(f"Running: {cmd}\n")
!{cmd}

## 5️⃣ Single Image Translation

Translate text from a single image using multimodal TranslateGemma.

In [ ]:
from google.colab import files
from PIL import Image
import sys
import urllib.request
import os

# Add examples directory to path
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Configuration: Choose image source
USE_DEFAULT_IMAGE = True  # Set to False to upload your own image
DEFAULT_IMAGE_URL = "https://cdn.odigo.net/f91b9c108a1e0cd1117e1c46ee36eeca.jpg"

# Language configuration
SOURCE_LANG = "ja"  # This is a Japanese menu image

# Get image
if USE_DEFAULT_IMAGE:
    print(f"📥 Downloading default image from:\n   {DEFAULT_IMAGE_URL}\n")
    image_file = "demo_image.jpg"
    urllib.request.urlretrieve(DEFAULT_IMAGE_URL, image_file)
    print(f"✅ Downloaded: {image_file}")
else:
    print("📤 Please upload your image:")
    uploaded = files.upload()
    image_file = list(uploaded.keys())[0]
    print(f"\n✅ Uploaded: {image_file}")

# Load backend
from transformers_multimodal_backend import TransformersMultimodalBackend

print("\n🔄 Loading multimodal backend...")
backend = TransformersMultimodalBackend()
backend.load_model()

# Translate
print(f"\n🔄 Translating {image_file}...")
print(f"Source language: {SOURCE_LANG} → Target language: {TARGET_LANG}")
result = backend.translate_image(image_file, source_lang=SOURCE_LANG, target_lang=TARGET_LANG)

# Display result
print(f"\n✅ Translation:")
print(result['translation'])
print(f"\n⏱️  Time: {result['time']:.2f}s, Speed: {result['metadata']['tokens_per_second']:.1f} tok/s")

## 6️⃣ Website Article Translation (Web Scraping)

Extract text from websites and translate them accurately using web scraping instead of screenshots.

In [ ]:
# Install web scraping dependencies
!pip install beautifulsoup4 requests -q

import requests
from bs4 import BeautifulSoup
import sys
import time
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Configuration
ARTICLE_URL = "https://aismiley.co.jp/ai_news/gemma3-rag-api-local-use/"
SOURCE_LANG = "ja"  # Japanese article

def extract_article_text(url):
    """Extract main article content from webpage"""
    print(f"🌐 Fetching webpage: {url}\n")
    
    # Fetch webpage
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    response = requests.get(url, headers=headers)
    response.encoding = response.apparent_encoding
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract title
    title = soup.find('h1')
    title_text = title.get_text(strip=True) if title else "No title found"
    
    # Extract main content - try common article selectors
    article_selectors = [
        'article',
        '.article-content',
        '.post-content',
        '.entry-content',
        'main'
    ]
    
    article_content = None
    for selector in article_selectors:
        article_content = soup.select_one(selector)
        if article_content:
            break
    
    if not article_content:
        article_content = soup.find('body')
    
    # Extract paragraphs
    paragraphs = []
    for p in article_content.find_all(['p', 'h2', 'h3']):
        text = p.get_text(strip=True)
        if len(text) > 20:  # Skip short paragraphs (likely navigation/ads)
            paragraphs.append(text)
    
    # Combine text
    full_text = f"{title_text}\n\n" + "\n\n".join(paragraphs[:15])  # First 15 paragraphs
    
    return {
        'title': title_text,
        'text': full_text,
        'paragraph_count': len(paragraphs)
    }

# Extract article
print("📄 Extracting article content...\n")
article = extract_article_text(ARTICLE_URL)

print(f"✅ Extracted article:")
print(f"   Title: {article['title']}")
print(f"   Paragraphs: {article['paragraph_count']}")
print(f"   Text length: {len(article['text'])} characters\n")

# Load translation backend
from transformers_backend import TransformersBackend

print("🔄 Loading translation backend...")
backend = TransformersBackend()
backend.load_model()

# Translate article
print(f"\n🔄 Translating article...")
print(f"Source language: {SOURCE_LANG} → Target language: {TARGET_LANG}\n")

start_time = time.time()
result = backend.translate(article['text'], source_lang=SOURCE_LANG, target_lang=TARGET_LANG)
end_time = time.time()

# Display result with word wrap
import textwrap

print(f"\n✅ Translation Result:")
print("=" * 80)
wrapped_translation = textwrap.fill(result['translation'], width=80, break_long_words=False, break_on_hyphens=False)
print(wrapped_translation)
print("=" * 80)

print(f"\n⏱️  Time: {result['time']:.2f}s")
print(f"📊 Speed: {result['metadata']['tokens_per_second']:.1f} tok/s")
print(f"🔤 Tokens: {result['tokens']} (input: {result['metadata']['input_tokens']}, output: {result['metadata']['output_tokens']})")

## 7️⃣ Website Screenshot Translation

Capture a screenshot of any website and translate it to Traditional Chinese.

In [ ]:
# Install system dependencies for Chromium
!apt-get update -qq
!apt-get install -y -qq libatk1.0-0 libatk-bridge2.0-0 libcups2 libxkbcommon0 libxcomposite1 libxdamage1 libxrandr2 libgbm1 libpango-1.0-0 libcairo2 libasound2

# Install Playwright
!pip install playwright -q
!playwright install chromium --with-deps

# Screenshot website and translate
import asyncio
from playwright.async_api import async_playwright
from PIL import Image
import sys
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Configuration
WEBSITE_URL = "https://www.yomiuri.co.jp/national/20260117-GYT1T00119/"
SOURCE_LANG = "ja"  # Japanese news website

async def capture_screenshot(url):
    """Capture website screenshot using Playwright async API"""
    print(f"📸 Capturing screenshot of: {url}\n")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page(viewport={'width': 1280, 'height': 1024})
        await page.goto(url, wait_until='networkidle')
        await page.screenshot(path='website_screenshot.png', full_page=False)
        await browser.close()
    
    print("✅ Screenshot saved: website_screenshot.png\n")

# Capture screenshot
await capture_screenshot(WEBSITE_URL)

# Load backend
from transformers_multimodal_backend import TransformersMultimodalBackend

print("🔄 Loading multimodal backend...")
backend = TransformersMultimodalBackend()
backend.load_model()

# Translate
print(f"\n🔄 Translating screenshot...")
print(f"Source language: {SOURCE_LANG} → Target language: {TARGET_LANG}\n")
result = backend.translate_image('website_screenshot.png', source_lang=SOURCE_LANG, target_lang=TARGET_LANG)

# Display result
print(f"\n✅ Translation:")
print(result['translation'])
print(f"\n⏱️  Time: {result['time']:.2f}s, Speed: {result['metadata']['tokens_per_second']:.1f} tok/s")

# Display screenshot
from IPython.display import Image as IPImage, display
display(IPImage('website_screenshot.png', width=800))

## 📝 Notes

- **Backend**: `transformers` is best for Colab GPU (T4)
- **Target Language**: Default is `zh-TW` (Traditional Chinese), change in Configuration section
- **Image Mode**: Slower but preserves visual context (tables, charts, layout)
- **DPI**: Lower DPI (72-96) is faster, higher DPI (150) has better quality

## 🔗 Links

- [GitHub Repository](https://github.com/jimmyliao/trans-gemma)
- [TranslateGemma Model](https://huggingface.co/google/translategemma-4b-it)
- [Documentation](https://github.com/jimmyliao/trans-gemma/blob/main/examples/README.md)